In [1]:
import os

In [2]:
pwd

'd:\\MLOPS\\Project_MLOPS\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'd:\\MLOPS\\Project_MLOPS'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from src.mlProject import logger
from src.mlProject.utils.common import get_size


In [17]:
class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        try:
            if not Path(self.config.local_data_file).exists():
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                logger.info(f"{filename} downloaded with the following info:\n{headers}")
            else:
                logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
        except (error.URLError, error.HTTPError) as e:
            logger.error(f"Error downloading file: {e}")

    def extract_zip_file(self):
        unzip_path = Path(self.config.unzip_dir)
        os.makedirs(unzip_path, exist_ok=True)
        
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
        except zipfile.BadZipFile:
            logger.error(f"The file {self.config.local_data_file} is not a valid ZIP file.")
        except Exception as e:
            logger.error(f"Error extracting ZIP file: {e}")


  

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"An error occurred: {e}")
    raise e

[2024-02-05 13:55:27,202: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-05 13:55:27,208: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-05 13:55:27,209: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-05 13:55:27,214: INFO: common: created directory at: artifacts]
[2024-02-05 13:55:27,216: INFO: common: created directory at: artifacts/data_ingestion]


[2024-02-05 13:55:28,921: INFO: 3677038119: artifacts/data_ingestion/data.zip downloaded with the following info:
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1F84:1D450A:25BDB8:3A63CE:65C09B78
Accept-Ranges: bytes
Date: Mon, 05 Feb 2024 08:25:29 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100035-HYD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1707121529.815933,VS0,VE361
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: ad3984c369277acb3ccd59b59a9d4699f7e92973
Expires: Mon, 05 Feb 2024 08:30:29 GMT
Source-Age: 0

]
